---
## Section 6: Practice Exercise

### 🎯 Challenge: Build a Content Creation Pipeline

**Objective:** Create a collaborative multi-agent system that transforms raw ideas into polished blog posts.

### Requirements

Build a **3-agent pipeline** using the collaborative (handoff) pattern:

1. **Brainstormer Agent**
   - Tool: `generate_ideas(topic: str) -> str`
   - Generates 3-5 key points about the topic
   - Returns bullet points of ideas

2. **Writer Agent**  
   - Tool: `draft_content(ideas: str) -> str`
   - Takes ideas and writes a draft blog post
   - Returns structured content with intro, body, conclusion

3. **Editor Agent**
   - Tool: `improve_writing(draft: str) -> str`
   - Polishes the draft for clarity and flow
   - Returns final polished version

### Implementation Steps

1. **Define your tools** - Create the 3 tools above (simulated output is fine)
2. **Create custom state** - What fields do you need to pass between agents?
3. **Build the 3 agents** - Each with `llm.bind_tools([...])` and tool invocation logic
4. **Construct the graph** - Linear pipeline: brainstormer → writer → editor
5. **Test it** - Try: "Create a blog post about machine learning"

### Success Criteria

✅ All 3 agents have tools bound  
✅ State passes information through the pipeline  
✅ Each agent invokes its tool  
✅ Final output is polished content  
✅ Graph compiles and runs without errors

### Bonus Challenges

- **Add a 4th agent** - Fact-checker that validates claims

### Tips

- Start by copying the collaborative pattern structure
- Keep tools simple - they can return hardcoded examples
- Test each agent individually before connecting them
- Use print statements to see the pipeline flow

**Do it yourself. Good luck!** 🚀

In [1]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [5]:
@tool
def generate_ideas(topic: str) -> str:
    """Brainstorm 3-5 key points for a blog post."""
    points = [
        f"Key point {i+1} about {topic}" for i in range(5)
    ]
    return "\n".join(points)

@tool
def draft_content(ideas: str) -> str:
    """Write a draft blog post from key points."""
    return (
        f"Introduction:\nThis blog explores the topic: {ideas.splitlines()[0]}...\n\n"
        f"Body:\n" + "\n".join([f"{i+1}. {point}" for i, point in enumerate(ideas.splitlines())]) + "\n\n"
        "Conclusion:\nThese points summarize the key takeaways."
    )

@tool
def improve_writing(draft: str) -> str:
    """Polish draft for clarity and flow."""
    polished = draft.replace("Key point", "Important point")
    polished = polished.replace("\n\n", "\n")  # Simplified formatting
    return f"📝 Polished Blog Post:\n{polished}"

In [6]:
class BlogState(TypedDict):
    messages: list
    task: str
    ideas: str
    draft: str
    polished: str
    fact_checked: str


# --- Brainstormer Agent ---
brainstormer_prompt = SystemMessage(content="""
You are a creative Brainstormer.
Use generate_ideas tool to create 3-5 key points for the topic.
""")
brainstormer_llm = llm.bind_tools([generate_ideas])

def brainstormer_agent(state: BlogState) -> dict:
    topic = state["task"]
    print("💡 Brainstormer: Generating ideas...")
    ideas = generate_ideas.invoke({"topic": topic})
    print(f"   Ideas:\n{ideas}\n")
    return {"messages": [AIMessage(content=ideas)], "ideas": ideas}

# --- Writer Agent ---
writer_prompt = SystemMessage(content="""
You are a Writer Agent.
Use draft_content tool to write a full blog post from ideas.
""")
writer_llm = llm.bind_tools([draft_content])

def writer_agent(state: BlogState) -> dict:
    ideas = state["ideas"]
    print("✍️ Writer: Drafting blog post...")
    draft = draft_content.invoke({"ideas": ideas})
    print(f"   Draft created.\n")
    return {"messages": [AIMessage(content=draft)], "draft": draft}

# --- Editor Agent ---
editor_prompt = SystemMessage(content="""
You are an Editor Agent.
Use improve_writing tool to polish the draft.
""")
editor_llm = llm.bind_tools([improve_writing])

def editor_agent(state: BlogState) -> dict:
    draft = state["draft"]
    print("📝 Editor: Polishing draft...")
    polished = improve_writing.invoke({"draft": draft})
    print(f"   Draft polished.\n")
    return {"messages": [AIMessage(content=polished)], "polished": polished}

print("✅ Agents created")


✅ Agents created


In [ ]:
graph_builder = StateGraph(BlogState)

graph_builder.add_node("brainstormer", brainstormer_agent)
graph_builder.add_node("writer", writer_agent)
graph_builder.add_node("editor", editor_agent)
graph_builder.add_node("fact_checker", fact_checker_agent)

# Linear pipeline
graph_builder.add_edge(START, "brainstormer")
graph_builder.add_edge("brainstormer", "writer")
graph_builder.add_edge("writer", "editor")
graph_builder.add_edge("editor", "fact_checker")
graph_builder.add_edge("fact_checker", END)

blog_pipeline = graph_builder.compile()
print("✅ Blog post pipeline compiled")

NameError: name 'fact_checker_agent' is not defined

In [ ]:
result = blog_pipeline.invoke({
    "task": "Machine Learning and Multi-Agent Systems",
    "messages": []
})

print("\n" + "="*70)
print("FINAL BLOG POST OUTPUT:")
print("="*70)
print(result["fact_checked"])
print("="*70)